In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
app_df = pd.read_csv("./Resources/charity_data.csv")
app_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
app_df=app_df.drop(columns=['EIN', 'NAME'])
app_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Look at APPLICATION_TYPE value counts for binning
type_counts = app_df.APPLICATION_TYPE.value_counts()
type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Determine which values to replace if counts are less than ...?
replace_app = list(type_counts[type_counts<500].index)

# Replace in dataframe
for app in replace_app:
    app_df.APPLICATION_TYPE = app_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
app_df.APPLICATION_TYPE.value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Look at CLASSIFICATION value counts for binning
class_counts=app_df.CLASSIFICATION.value_counts()
class_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [10]:
# Determine which values to replace if counts are less than ..?
replace_class=list(class_counts[class_counts<500].index)

# Replace in dataframe
for cls in replace_class:
    app_df.CLASSIFICATION = app_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
app_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [11]:
# Generate our categorical variable lists
app_cat = app_df.dtypes[app_df.dtypes=='object'].index.tolist()

app_df[app_cat].nunique()

APPLICATION_TYPE          6
AFFILIATION               6
CLASSIFICATION            7
USE_CASE                  5
ORGANIZATION              4
INCOME_AMT                9
SPECIAL_CONSIDERATIONS    2
dtype: int64

In [12]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df=pd.DataFrame(enc.fit_transform(app_df[app_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(app_cat)
encode_df.head()

c:\Users\jakat\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [13]:
# Merge one-hot encoded features and drop the originals
app_df=app_df.merge(encode_df, left_index=True, right_index=True)

app_df=app_df.drop(app_cat, 1)
app_df.head()

c:\Users\jakat\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [14]:
# Split our preprocessed data into our features and target arrays
y=app_df.IS_SUCCESSFUL
X=app_df.drop(columns=["IS_SUCCESSFUL"])

# Split the preprocessed data into a training and testing dataset
X_train,X_test, y_train, y_test=train_test_split(X,y, random_state=42, stratify=y)

In [15]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [16]:
#Experiments to improve accuracy
#TEST 1: Lower neurons, but add a layer
features=len(X_train_scaled[0])
hidden_nodes1 = 40
hidden_nodes2 = 15

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes1, input_dim=features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes2, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units = 5, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define checkpoint path and the filenames to use
os.makedirs("checkpoints_optimized/", exist_ok=True)
checkpoint_path = "checkpoints_optimized/weights.{epoch:02d}.hdf5"

#Create a callback to save the model's weights
cp_callback=ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    period=5
)

# Train the model
fit_model=nn.fit(X_train_scaled,y_train, epochs=100, callbacks =[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss:.3f}, Accuracy: {model_accuracy: .3f}")



Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5999 - accuracy: 0.6874
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5615 - accuracy: 0.7243
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5575 - accuracy: 0.7261
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5557 - accuracy: 0.7275
Epoch 5/100
759/804 [===========================>..] - ETA: 0s - loss: 0.5529 - accuracy: 0.7274
Epoch 5: saving model to checkpoints_optimized\weights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5535 - accuracy: 0.7273
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5519 - accuracy: 0.7276
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5507 - accuracy: 0.7270
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5503 - accuracy: 0.7284
Epoch 9/100
804/804 [=======================

In [17]:
#TEST 2: Keep same neuron count, add a layer
features=len(X_train_scaled[0])
hidden_nodes1 = 80
hidden_nodes2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes1, input_dim=features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes2, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define checkpoint path and the filenames to use
os.makedirs("checkpoints_optimized/", exist_ok=True)
checkpoint_path = "checkpoints_optimized/weights.{epoch:02d}.hdf5"

#Create a callback to save the model's weights
cp_callback=ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    period=5
)

# Train the model
fit_model=nn.fit(X_train_scaled,y_train, epochs=100, callbacks =[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss:.3f}, Accuracy: {model_accuracy: .3f}")

Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.5750 - accuracy: 0.7173
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5559 - accuracy: 0.7314
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5533 - accuracy: 0.7318
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5521 - accuracy: 0.7322
Epoch 5/100
777/804 [===========================>..] - ETA: 0s - loss: 0.5511 - accuracy: 0.7333
Epoch 5: saving model to checkpoints_optimized\weights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5505 - accuracy: 0.7339
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5493 - accuracy: 0.7341
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5491 - accuracy: 0.7337
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5485 - accuracy: 0.7348
Epoch 9/100
804/804 [=======================

In [18]:
#TEST 3: change the optimizer
# Not much changed.  Adamax was the best
from tensorflow.keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adamax, Nadam
features=len(X_train_scaled[0])
hidden_nodes1 = 80
hidden_nodes2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes1, input_dim=features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes2, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adamax', metrics=['accuracy'])

#Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define checkpoint path and the filenames to use
os.makedirs("checkpoints_optimized/", exist_ok=True)
checkpoint_path = "checkpoints_optimized/weights.{epoch:02d}.hdf5"

#Create a callback to save the model's weights
cp_callback=ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    period=5
)

# Train the model
fit_model=nn.fit(X_train_scaled,y_train, epochs=100, callbacks =[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss:.3f}, Accuracy: {model_accuracy: .3f}")

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5763 - accuracy: 0.7231
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5583 - accuracy: 0.7302
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5546 - accuracy: 0.7307
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5524 - accuracy: 0.7323
Epoch 5/100
791/804 [============================>.] - ETA: 0s - loss: 0.5508 - accuracy: 0.7319
Epoch 5: saving model to checkpoints_optimized\weights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5510 - accuracy: 0.7318
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5502 - accuracy: 0.7326
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5491 - accuracy: 0.7345
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5490 - accuracy: 0.7343
Epoch 9/100
804/804 [=======================

In [19]:
#TEST 4: Change the activations
features=len(X_train_scaled[0])
hidden_nodes1 = 80
hidden_nodes2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes1, input_dim=features, activation='tanh'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes2, activation='gelu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adamax', metrics=['accuracy'])

#Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define checkpoint path and the filenames to use
os.makedirs("checkpoints_optimized/", exist_ok=True)
checkpoint_path = "checkpoints_optimized/weights.{epoch:02d}.hdf5"

#Create a callback to save the model's weights
cp_callback=ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    period=5
)

# Train the model
fit_model=nn.fit(X_train_scaled,y_train, epochs=100, callbacks =[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss:.3f}, Accuracy: {model_accuracy: .3f}")

Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.5802 - accuracy: 0.7142
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5606 - accuracy: 0.7284
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5570 - accuracy: 0.7303
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5550 - accuracy: 0.7325
Epoch 5/100
792/804 [============================>.] - ETA: 0s - loss: 0.5537 - accuracy: 0.7317
Epoch 5: saving model to checkpoints_optimized\weights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5538 - accuracy: 0.7318
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5523 - accuracy: 0.7332
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5514 - accuracy: 0.7332
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5508 - accuracy: 0.7335
Epoch 9/100
804/804 [=======================

In [20]:
#TEST 5:  Adding a fourth layer...
features=len(X_train_scaled[0])
hidden_nodes1 = 80
hidden_nodes2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes1, input_dim=features, activation='tanh'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes2, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation='gelu'))

# Fourth Layer
nn.add(tf.keras.layers.Dense(units=5, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define checkpoint path and the filenames to use
os.makedirs("checkpoints_optimized/", exist_ok=True)
checkpoint_path = "checkpoints_optimized/weights.{epoch:02d}.hdf5"

#Create a callback to save the model's weights
cp_callback=ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    period=5
)

# Train the model
fit_model=nn.fit(X_train_scaled,y_train, epochs=100, callbacks =[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss:.3f}, Accuracy: {model_accuracy: .3f}")

Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.5692 - accuracy: 0.7224
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5550 - accuracy: 0.7302
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5528 - accuracy: 0.7313
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5513 - accuracy: 0.7325
Epoch 5/100
774/804 [===========================>..] - ETA: 0s - loss: 0.5501 - accuracy: 0.7334
Epoch 5: saving model to checkpoints_optimized\weights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5502 - accuracy: 0.7332
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5495 - accuracy: 0.7331
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5488 - accuracy: 0.7341
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5483 - accuracy: 0.7341
Epoch 9/100
804/804 [=======================

In [21]:
# Save and Export
nn.save("AlphabetSoupCharity_Optimization.h5")